In [1]:
import matplotlib.pyplot as plt
import yaml
import plotly.express as px
import pandas as pd
import numpy as np

import utils

In [145]:
data = yaml.load(
    open("demo.yaml", "r"), 
    Loader=yaml.FullLoader
)

In [172]:
def inverse_mil(mil):
    new = mil + 3200
    return new if new < 6400 else new - 6400
def get_delta_matrix(input:list):
    arr = np.array(input)
    delta_matrix = np.tril(arr[:,np.newaxis] - arr)
    return delta_matrix

In [173]:
def get_easting_northing_matrix(data):
    easting = []
    northing = []
    labels = []

    for platoon_name, grids in data.items():
        for platoon_element, grid in grids.items():
            e, n = utils.get_easting_northing(grid)
            easting.append(e)
            northing.append(n)
            labels.append(platoon_element)
            
    return get_delta_matrix(easting), get_delta_matrix(northing), labels

In [174]:
dx, dy, labels = get_easting_northing_matrix(data)

In [175]:
def get_range_table(dx,dy,labels):
    range_matrix = np.sqrt(np.square(dx) + np.square(dy)).astype(int)
    return range_matrix#, index=labels, columns=labels)

In [176]:
# def get_mils_table(dx,dy):
#     mils_matrix = 

In [177]:
arr_ = np.divide(dx,dy,out=np.zeros_like(dx,dtype=float), where=dy!=0)
arr = np.transpose(arr_)

In [190]:
ax = np.zeros((35,35))
for i in range(35):
    for j in range(35):
        # flip
        x = dx[i][j]
        y = dy[i][j]
        
        if (x==0) and (y==0):
            continue
        elif (x==0):
            ax[i][j] = 6400 if y > 0 else 3200
        elif (y==0):
            ax[i][j] = 1600 if x > 0 else 4800
        else:
             
            mil = utils.get_mil(x,y)
            ax[i][j] = mil if mil < 6400 else inverse_mil(mil)
axx = np.transpose(ax.astype(int))
#pd.DataFrame(axx)

In [191]:
def _color_red_or_green(val):
    if val < 0: color = 'red'
    elif val > 0: color = "green"
    else: color = "black"
    return 'background-color: %s' % color

In [192]:
print(dof+100,dof-100)
print(inverse_mil(dof+100),inverse_mil(dof-100))

2400 2200
5600 5400


In [193]:
df = pd.DataFrame(get_range_table(dx,dy,labels) + axx, index=labels, columns=labels)

In [194]:
dof = 2300
mask = ((df['t3_fp2'] > (dof-100)) & (df['t3_fp2'] < (dof+100))) | ((df['t3_fp2'] < inverse_mil(dof+100)) & (df['t3_fp2'] > inverse_mil(dof-100)))
df['t3_fp2'][mask] =  - df['t3_fp2'][mask]


In [195]:
df.style.applymap(_color_red_or_green)

,brv,bcp,css_ha,plt1_prv,plt1_pcp,plt1_rp1,plt1_rp2,t1_fp1,t1_fp2,t1_hp1,t1_hp2,t2_fp1,t2_fp2,t2_hp1,t2_hp2,t3_fp1,t3_fp2,t3_hp1,t3_hp2,plt2_prv,plt2_pcp,plt2_rp1,plt2_rp2,t4_fp1,t4_fp2,t4_hp1,t4_hp2,t5_fp1,t5_fp2,t5_hp1,t5_hp2,t6_fp1,t6_fp2,t6_hp1,t6_hp2
brv,0,1465,1549,4145,4092,4231,3922,3892,3757,3908,3774,4124,4031,4118,4039,4340,4127,4372,4267,4145,4328,4250,4271,3905,3891,4077,4089,3729,3672,4048,3918,3522,3304,3824,3565
bcp,151,0,1600,3491,3717,3961,3592,3672,3530,3655,3500,3940,3832,3903,3787,4206,3894,4207,4074,4339,4412,4334,4333,3994,3993,4150,4171,3864,3840,4149,4059,3735,3581,4025,3884
css_ha,400,250,0,4082,3561,3200,3469,3238,3285,3254,3403,3528,3405,3401,3225,3887,3337,3745,3565,4519,4527,4452,4428,4127,4141,4261,4291,4054,4062,4290,4240,4009,3936,4256,4210
plt1_prv,250,177,262,0,4000,4328,3672,3746,3546,3738,3504,4113,3967,4100,3954,4435,4113,4524,4354,4800,4715,4585,4524,4137,4157,4303,4346,4044,4054,4355,4301,3974,3846,4346,4303
plt1_pcp,390,308,287,141,0,1600,3200,3626,3355,3565,3200,4164,3952,4163,3915,4589,4201,1600,4572,5001,4873,4736,4648,4263,4300,4422,4479,4218,4262,4509,4490,4231,4188,4585,4632
plt1_rp1,471,367,270,223,100,0,4000,3275,3355,3327,3626,3908,3635,3762,3278,4517,3672,1600,4398,4968,4864,4743,4662,4305,4344,4452,4507,4278,4328,4538,4527,4314,4301,4617,4664
plt1_rp2,460,399,383,223,100,141,0,3826,3423,3855,3200,4485,4250,4632,4529,1546,1600,1226,1372,5188,5015,4864,4749,4344,4395,4507,4575,4328,4398,4622,4627,4398,4412,4764,4873
t1_fp1,668,603,540,430,295,270,208,0,4122,5818,5165,983,1031,412,6327,957,178,487,435,5348,5179,5039,4909,4529,4599,4673,4754,4580,4687,4821,4862,4749,4865,5021,5161
t1_fp2,673,627,597,450,328,328,230,89,0,201,5928,961,978,599,304,949,426,584,564,5470,5272,5119,4968,4562,4642,4715,4805,4632,4761,4885,4942,4848,5013,5130,5306
t1_hp1,585,521,470,347,213,201,125,83,127,0,4472,1484,4638,1001,472,1218,655,760,783,5290,5118,4973,4847,4456,4519,4608,4685,4482,4577,4745,4774,4618,4704,4928,5062
